In [7]:
# Class: COEN 140 Lab Machine Learning/Data Mining
# Name: Matthew Davenport
# Date: 9/25/2022
# Title: Lab 4 – Iris classifier LDA and QDA without python libraries
# Description: This program will perform clssifications
#           on iris datasets based on the sepal
#           lengths/widths and petal lengths/widths and 
#           classify them as Setosa, Versicolour, or Virginica
#           without using python ML libraries

import numpy as np
import math

ROWS = 150
COLUMNS = 5
CLASSIFICATIONS = 3
FEATURES = ['sepal length','sepal width', 'pedal length', 'pedal width']

In [8]:
# for testing of proper reading of data (provided)
def test_dataset(data):
    if len(data) != 150:
        return False
    for row in data:
        if len(row) != 5:
            return False
        for column in row[:-1]:
            if type(column) != np.float64:
                return False
            
        if type(row[-1]) != str:
            return False
    
    return True

In [9]:
# read the file and convert the contents to the proper types and return list of lists
def read_file(): 
    data = [[] for x in range(ROWS)]                 # initialize empty list of lists length 150
    file = open('C:/Users/daven/COEN140/Lab3/test_dataset.txt', 'r')            # open file as read
    contents = [line.split() for line in file]      # put contents of file into list of lists of characters
    
    #iterate through contents stopping at each , in the list of characters to convert the first 4 elements into 
    #           np.float64 nd the last into a string to be loaded into data
    for row in range(len(contents)):                # iterate through contents
        i = 0
        for s in contents[row][0].split(","):       # usingn , as delimiter
            if i == (COLUMNS - 1):                              # at the last element just append as string
                data[row].append(s)
                break
            i += 1
            data[row].append(np.float64(s))         # append into data[row] the number converted to np.float64
    return data

In [10]:
# test the actual classifications compared to our predictions for QDA and LDA
# returns the percent error of predictions
def classification_compare(predicted, actuals):
    length = len(actuals)
    correct_predictions = 0
    for i in range(length):
        if predicted[i] == actuals[i]:
            correct_predictions += 1
        else:
            print("\nIncorrect prediction at index " + str(i))
            print("Predicted: " + predicted[i])
            print("Actual: ", actuals[i])
        
    return 100 - correct_predictions/length * 100

In [30]:
def category_means(category): 
    means = []
    for i in range(4): 
        temp_mean = 0
        for j in range(40):
            temp_mean += category[j][i]
        temp_mean = temp_mean/40
        means.append(temp_mean)
    return means
            

In [37]:
def variance(category, means):
    variances = []
    for i in range(4):
        new_arr = np.array([observation[i] for observation in category])
        variances.append(np.var(new_arr))
        
    return variances

In [41]:
def norm_pdf(X, u, var):
    prob_density = (np.pi*var) * np.exp(-0.5*((X-u)/var)**2)
    return prob_density

In [11]:
# Testing contentes of file converted successfully 
data = read_file()
print(test_dataset(data))

# split into training and testing lists 
training = data[:40:] + data[50:90:] + data[100:140:]
testing = data[40:50:] + data[90:100:] + data[140:150:]

# ensure proper length for training and testing items
print(len(training))
print(len(testing))

# using np array to better manipulate columns of data
training_array = np.array(training)
testing_array = np.array(testing)

True
120
30


In [29]:
# separation of X and y and testing/training
# splitting of arrays into arrays by classification of flower
# convert X from strings to floats using numpy
x_training = training_array[:,0:4].astype(np.float64)
x_testing = testing_array[:,0:4].astype(np.float64)
y_testing = testing_array[:,4]
y_training = training_array[:,4]

# splitting of arrays based on classification
setosa_training = x_training[:40]
versicolor_training = x_training[40:80]
virginica_training = x_training[80:120]

# check arrays were split correctly 
print("Arrays split sucessfully? " + 
      str(np.allclose(np.concatenate((setosa_training, np.concatenate((versicolor_training, virginica_training)))),x_training)))




Arrays split sucessfully? True


In [48]:
# computing means for each cateory
setosa_means = category_means(setosa_training)
versicolor_means = category_means(versicolor_training)
virginica_means = category_means(virginica_training)
# print(setosa_means)
# print(versicolor_means)
# print(virginica_means)

# # variances
# setosa_variances = variance(setosa_training, setosa_means)
# versicolor_variances = variance(versicolor_training, versicolor_means)
# virginica_variances = variance(virginica_training, virginica_means)
# print(setosa_variances)
# print(versicolor_variances)
# print(virginica_variances)

# covariances for each classification
setosa_cov = np.cov(setosa_training, rowvar=False)
versicolor_cov = np.cov(versicolor_training, rowvar=False)
virginica_cov = np.cov(virginica_training, rowvar=False)
print(setosa_cov, end="\n\n")
print(versicolor_cov, end="\n\n")
print(virginica_cov, end="\n\n")

[[0.13112179 0.09897436 0.01298077 0.01362179]
 [0.09897436 0.13271795 0.00205128 0.0145641 ]
 [0.01298077 0.00205128 0.02958333 0.00458333]
 [0.01362179 0.0145641  0.00458333 0.00994231]]

[[0.27374359 0.08661538 0.17212821 0.05230769]
 [0.08661538 0.11087179 0.08087179 0.04538462]
 [0.17212821 0.08087179 0.20353205 0.07371795]
 [0.05230769 0.04538462 0.07371795 0.04307692]]

[[0.46794231 0.11041026 0.35777564 0.05125641]
 [0.11041026 0.11323077 0.08107692 0.04625641]
 [0.35777564 0.08107692 0.34532692 0.05930769]
 [0.05125641 0.04625641 0.05930769 0.07425641]]



In [67]:
# QDA classification for test data
from scipy.stats import multivariate_normal

# compute probabilities of each entry belonging to category of flower for testing group
set_prob_test = multivariate_normal.pdf(x_testing, setosa_means, setosa_cov)
vers_prob_test = multivariate_normal.pdf(x_testing, versicolor_means, versicolor_cov)
virg_prob_test = multivariate_normal.pdf(x_testing, virginica_means, virginica_cov)
print(set_prob_test, end="\n\n")
print(vers_prob_test, end="\n\n")
print(virg_prob_test, end="\n\n")

# classify based on comparisons of probabilities that each belongs to a certain category
qda_test_class = []
for i in range(30): 
    if set_prob_test[i] > vers_prob_test[i] and set_prob_test[i] > virg_prob_test[i]:
        qda_test_class.append('Iris-setosa')
    elif vers_prob_test[i] > set_prob_test[i] and vers_prob_test[i] > virg_prob_test[i]:
        qda_test_class.append('Iris-versicolor')
    elif virg_prob_test[i] > set_prob_test[i] and virg_prob_test[i] > vers_prob_test[i]:
        qda_test_class.append('Iris-virginica')
    
print(qda_test_class == y_testing)    ## should all be true

[7.81177785e+000 1.81048629e-003 2.27890748e+000 5.59770776e-003
 1.00272575e-001 3.10718374e+000 3.02575674e+000 8.97886808e+000
 1.10596611e+001 1.61224185e+001 1.25995675e-075 2.35307034e-088
 6.17990203e-062 1.21308375e-037 1.94845153e-071 1.03616745e-065
 4.92882422e-070 4.70777807e-074 4.25477587e-034 2.42969159e-067
 2.20243455e-197 2.22033914e-167 8.96771811e-140 7.88258040e-203
 4.69068926e-209 4.04541604e-172 4.08462993e-138 6.88871015e-148
 2.77756948e-177 1.10142004e-130]

[5.88902311e-23 1.73532119e-12 1.70664995e-19 3.58520997e-18
 6.31867093e-21 2.01425700e-16 4.57657405e-26 5.79339655e-19
 1.12468803e-25 9.24900533e-21 2.40751868e-01 2.44508165e+00
 3.92526536e+00 3.67453462e-01 3.33068754e+00 4.88100376e-01
 2.71070381e+00 3.77762450e+00 9.19610454e-03 3.97986533e+00
 1.65374215e-08 1.09930077e-09 2.04944426e-03 1.00979403e-05
 2.91398336e-09 6.38674374e-09 2.35510388e-04 4.84677519e-03
 6.35435188e-06 7.32798211e-02]

[9.35547567e-36 4.79357840e-26 1.93466684e-29 6.64

In [57]:
## print(f"{100 * incorrect/total length:.2f}%") printing percentges
# computing QDA classifications for the training data

from scipy.stats import multivariate_normal

# probabilites
set_prob_train = multivariate_normal.pdf(x_training, setosa_means, setosa_cov)
vers_prob_train = multivariate_normal.pdf(x_training, versicolor_means, versicolor_cov)
virg_prob_train = multivariate_normal.pdf(x_training, virginica_means, virginica_cov)
print(set_prob_train, end="\n\n")
print(vers_prob_train, end="\n\n")
print(virg_prob_train, end="\n\n")

# classify based on comparisons of probabilities that each belongs to a certain category
qda_train_class = []
for i in range(120): 
    if set_prob_train[i] > vers_prob_train[i] and set_prob_train[i] > virg_prob_train[i]:
        qda_train_class.append('Iris-setosa')
    elif vers_prob_train[i] > set_prob_train[i] and vers_prob_train[i] > virg_prob_train[i]:
        qda_train_class.append('Iris-versicolor')
    elif virg_prob_train[i] > set_prob_train[i] and virg_prob_train[i] > vers_prob_train[i]:
        qda_train_class.append('Iris-virginica')
    
print(qda_train_class == y_testing)    ## should all be true


NameError: name 'set_prob_test' is not defined

In [ ]:
# LDA classifier
class LDA: 
    def __init__(self) -> None:
        pass
    
    def fit(X, y): 
        return 0
    
    def predict(X):
        return 0
   